# Multi-Agent Research System - Real Agent Demonstration

## Actual Agent Reasoning & Collaboration

This demonstration shows **real agents** that:
- Use LLM reasoning to analyze information
- Build upon each other's findings
- Collaborate through shared knowledge
- Generate original insights (not hardcoded responses)

**Agents:**
- **Market Research Agent** - Analyzes data and generates market insights
- **Consumer Insights Agent** - Reasons about user behavior and demand
- **Competitive Intelligence Agent** - Synthesizes competitive positioning
- **Research Coordinator** - Orchestrates and synthesizes findings

In [26]:
# Install dependencies
!pip install -q langchain langchain-openai langchain-community > /dev/null 2>&1
print("Dependencies installed for real agent demonstration")

Dependencies installed for real agent demonstration


In [27]:
import json
import uuid
import warnings
from typing import Dict, Any, List
from datetime import datetime

# LangChain imports
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Get API key
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
openai_api_key = user_secrets.get_secret("openai_api_key")

warnings.filterwarnings('ignore')
print("System ready for real agent demonstration")

System ready for real agent demonstration


In [28]:
class SharedKnowledgeBase:
    """Real-time knowledge sharing between agents"""
    
    def __init__(self):
        self.findings = {}
        self.agent_contributions = {}
        
    def add_finding(self, category: str, key: str, data: Dict[str, Any], agent_name: str):
        """Store agent findings for cross-agent access"""
        finding_id = f"{category}_{key}"
        
        self.findings[finding_id] = {
            "category": category,
            "key": key,
            "data": data,
            "agent": agent_name,
            "timestamp": datetime.now().isoformat()
        }
        
        if agent_name not in self.agent_contributions:
            self.agent_contributions[agent_name] = []
        self.agent_contributions[agent_name].append(finding_id)
    
    def get_findings_for_agent(self, requesting_agent: str) -> str:
        """Get relevant findings for agent to build upon"""
        context = "Previous agent findings:\n"
        
        for finding_id, finding in self.findings.items():
            if finding["agent"] != requesting_agent:
                context += f"- {finding['agent']}: {finding['key']} - {finding['data'].get('summary', 'Analysis complete')}\n"
        
        return context if len(self.findings) > 0 else "No previous findings available."
    
    def get_all_findings(self) -> Dict[str, Any]:
        """Get all findings for final synthesis"""
        return self.findings

# Initialize knowledge base
shared_kb = SharedKnowledgeBase()
print("Shared knowledge base ready for agent collaboration")

Shared knowledge base ready for agent collaboration


In [29]:
class MarketResearchAgent:
    """Agent that reasons about market data and generates insights"""
    
    def __init__(self, openai_api_key: str, shared_kb: SharedKnowledgeBase):
        self.name = "Market Research Agent"
        self.shared_kb = shared_kb
        
        self.llm = ChatOpenAI(
            temperature=0.7,  # Allow creative analysis
            api_key=openai_api_key,
            model_name="gpt-4"
        )
        
        self.tools = self._create_tools()
        self._initialize_agent()
        
    def _create_tools(self):
        """Tools that provide data for agent to analyze"""
        
        def get_market_data(industry: str) -> str:
            """Retrieve raw market data for analysis"""
            # Simulate data sources - in real version would be APIs
            data_points = {
                "ai_software": {
                    "current_market_size_billion": 62.5,
                    "projected_size_2030_billion": 1400,
                    "key_segments": ["Machine Learning Platforms", "Natural Language Processing", "Computer Vision", "Robotic Process Automation"],
                    "regional_breakdown": {"North America": 0.42, "Europe": 0.28, "Asia Pacific": 0.22, "Other": 0.08},
                    "adoption_indicators": ["Enterprise digital transformation initiatives", "Remote work acceleration", "Data analytics demand", "Automation cost pressures"]
                }
            }
            
            key = industry.lower().replace(" ", "_")
            if key in data_points:
                return json.dumps(data_points[key], indent=2)
            return f"No data available for {industry}"
        
        def get_trend_indicators(sector: str) -> str:
            """Get trend indicators for analysis"""
            indicators = {
                "ai_software": {
                    "emerging_technologies": ["Generative AI", "Edge AI", "Multimodal AI", "AI Governance Tools"],
                    "market_shifts": ["From custom to SaaS solutions", "Increased focus on AI safety", "Integration with existing workflows"],
                    "investment_trends": ["$50B+ in AI funding in 2023", "Enterprise AI adoption accelerating", "Open source AI gaining traction"]
                }
            }
            
            key = sector.lower().replace(" ", "_")
            if key in indicators:
                return json.dumps(indicators[key], indent=2)
            return f"No trend data for {sector}"
        
        return [
            Tool(
                name="get_market_data",
                description="Retrieve market size, segments, and regional data for analysis",
                func=get_market_data
            ),
            Tool(
                name="get_trend_indicators",
                description="Get trend indicators and market shift data for analysis",
                func=get_trend_indicators
            )
        ]
    
    def _initialize_agent(self):
        """Initialize agent with reasoning capabilities"""
        
        system_prompt = """You are a Market Research Agent with expertise in analyzing market data and generating strategic insights.
        
Your capabilities:
- Analyze market size data and calculate growth rates
- Identify key market segments and their potential
- Assess regional market opportunities
- Synthesize trend indicators into strategic insights
- Consider broader economic and technology factors

When conducting research:
1. Use your tools to gather relevant data
2. Analyze the data with your expertise
3. Generate original insights and strategic implications
4. Consider risks and opportunities
5. Provide actionable recommendations

Always provide detailed analysis, not just data summaries."""
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        agent = create_openai_functions_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=prompt
        )
        
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,  # Show reasoning process
            handle_parsing_errors=True,
            max_iterations=3
        )
    
    def conduct_research(self, query: str) -> str:
        """Conduct research using agent reasoning"""
        # Get context from other agents
        context = self.shared_kb.get_findings_for_agent(self.name)
        
        enhanced_query = f"""{query}
        
Context from other agents:
{context}

Please conduct thorough analysis and provide strategic insights."""
        
        try:
            result = self.agent_executor.invoke({"input": enhanced_query})
            
            # Store findings for other agents
            self.shared_kb.add_finding(
                "market", 
                query[:50].replace(" ", "_"),
                {"summary": result["output"][:200] + "...", "full_analysis": result["output"]},
                self.name
            )
            
            return result["output"]
        except Exception as e:
            return f"Research analysis encountered an issue: {str(e)}"

# Initialize Market Research Agent
market_agent = MarketResearchAgent(openai_api_key, shared_kb)
print("Market Research Agent ready - will perform actual analysis")

Market Research Agent ready - will perform actual analysis


In [30]:
class ConsumerInsightsAgent:
    """Agent that analyzes consumer behavior and generates insights"""
    
    def __init__(self, openai_api_key: str, shared_kb: SharedKnowledgeBase):
        self.name = "Consumer Insights Agent"
        self.shared_kb = shared_kb
        
        self.llm = ChatOpenAI(
            temperature=0.7,
            api_key=openai_api_key,
            model_name="gpt-4"
        )
        
        self.tools = self._create_tools()
        self._initialize_agent()
        
    def _create_tools(self):
        """Tools providing consumer data for analysis"""
        
        def get_adoption_data(market: str) -> str:
            """Get consumer adoption metrics"""
            adoption_data = {
                "ai_software": {
                    "enterprise_adoption_rate": 0.23,
                    "smb_adoption_rate": 0.12,
                    "consumer_adoption_rate": 0.31,
                    "growth_rate_yoy": 0.41,
                    "usage_frequency": {"daily": 0.45, "weekly": 0.32, "monthly": 0.23},
                    "primary_use_cases": ["Content creation", "Data analysis", "Customer service", "Process automation"]
                }
            }
            
            key = market.lower().replace(" ", "_")
            if key in adoption_data:
                return json.dumps(adoption_data[key], indent=2)
            return f"No adoption data for {market}"
        
        def get_user_behavior_data(segment: str) -> str:
            """Get user behavior and preference data"""
            behavior_data = {
                "ai_users": {
                    "top_priorities": ["Accuracy and reliability", "Data privacy and security", "Ease of use", "Integration capabilities", "Cost effectiveness"],
                    "concern_areas": ["Data privacy (67%)", "Job displacement (45%)", "Over-reliance (38%)", "Bias and fairness (42%)"],
                    "decision_factors": ["Proven ROI", "Security compliance", "Vendor reputation", "Support quality"],
                    "satisfaction_drivers": ["Performance consistency", "Intuitive interface", "Responsive support", "Regular updates"]
                }
            }
            
            key = segment.lower().replace(" ", "_")
            if key in behavior_data:
                return json.dumps(behavior_data[key], indent=2)
            return f"No behavior data for {segment}"
        
        return [
            Tool(
                name="get_adoption_data",
                description="Get consumer adoption rates, growth, and usage patterns",
                func=get_adoption_data
            ),
            Tool(
                name="get_user_behavior_data",
                description="Get user preferences, concerns, and decision factors",
                func=get_user_behavior_data
            )
        ]
    
    def _initialize_agent(self):
        """Initialize consumer insights agent"""
        
        system_prompt = """You are a Consumer Insights Agent specializing in understanding user behavior, preferences, and market adoption patterns.
        
Your expertise includes:
- Analyzing adoption rates and identifying growth patterns
- Understanding user motivations and pain points
- Segmenting markets based on behavior and needs
- Identifying barriers to adoption and usage
- Translating user insights into business implications

When analyzing consumer data:
1. Look for patterns in adoption and usage behavior
2. Identify key user segments and their distinct needs
3. Analyze what drives user satisfaction and concerns
4. Consider the customer journey and experience gaps
5. Generate insights about market opportunities and risks

Build upon findings from other agents to provide comprehensive consumer perspective."""
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        agent = create_openai_functions_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=prompt
        )
        
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=3
        )
    
    def conduct_research(self, query: str) -> str:
        """Conduct consumer insights research"""
        # Get context from other agents
        context = self.shared_kb.get_findings_for_agent(self.name)
        
        enhanced_query = f"""{query}
        
Previous research context:
{context}

Please analyze consumer behavior and provide strategic insights."""
        
        try:
            result = self.agent_executor.invoke({"input": enhanced_query})
            
            # Store findings
            self.shared_kb.add_finding(
                "consumer",
                query[:50].replace(" ", "_"),
                {"summary": result["output"][:200] + "...", "full_analysis": result["output"]},
                self.name
            )
            
            return result["output"]
        except Exception as e:
            return f"Consumer analysis encountered an issue: {str(e)}"

# Initialize Consumer Insights Agent
consumer_agent = ConsumerInsightsAgent(openai_api_key, shared_kb)
print("Consumer Insights Agent ready - will analyze actual user behavior")

Consumer Insights Agent ready - will analyze actual user behavior


In [31]:
class CompetitiveIntelligenceAgent:
    """Agent that analyzes competitive landscape and positioning"""
    
    def __init__(self, openai_api_key: str, shared_kb: SharedKnowledgeBase):
        self.name = "Competitive Intelligence Agent"
        self.shared_kb = shared_kb
        
        self.llm = ChatOpenAI(
            temperature=0.7,
            api_key=openai_api_key,
            model_name="gpt-4"
        )
        
        self.tools = self._create_tools()
        self._initialize_agent()
        
    def _create_tools(self):
        """Tools providing competitive data for analysis"""
        
        def get_competitor_profiles(market: str) -> str:
            """Get competitive landscape data"""
            competitive_data = {
                "ai_software": {
                    "major_players": [
                        {"name": "OpenAI", "market_share_approx": 0.15, "key_products": ["GPT-4", "ChatGPT", "DALL-E"], "funding_billion": 11.3},
                        {"name": "Google", "market_share_approx": 0.12, "key_products": ["Bard", "Vertex AI", "Cloud AI"], "funding_billion": "N/A"},
                        {"name": "Microsoft", "market_share_approx": 0.11, "key_products": ["Copilot", "Azure AI", "Cognitive Services"], "funding_billion": "N/A"},
                        {"name": "Anthropic", "market_share_approx": 0.05, "key_products": ["Claude"], "funding_billion": 4.1}
                    ],
                    "market_concentration": "Fragmented - top 4 players control ~43% of market",
                    "entry_barriers": ["High compute costs", "Talent scarcity", "Data requirements", "Regulatory complexity"]
                }
            }
            
            key = market.lower().replace(" ", "_")
            if key in competitive_data:
                return json.dumps(competitive_data[key], indent=2)
            return f"No competitive data for {market}"
        
        def get_positioning_analysis(space: str) -> str:
            """Get market positioning and differentiation data"""
            positioning_data = {
                "ai_assistants": {
                    "differentiation_factors": ["Model capability", "Safety measures", "Enterprise features", "Cost structure", "Ecosystem integration"],
                    "competitive_moves": ["Price competition increasing", "Feature parity race", "Partnership strategies", "Vertical specialization"],
                    "white_space_opportunities": ["Industry-specific solutions", "Privacy-first alternatives", "Edge deployment", "Cost-effective options"],
                    "threat_assessment": {"new_entrants": "High", "substitutes": "Medium", "buyer_power": "Medium", "supplier_power": "High"}
                }
            }
            
            key = space.lower().replace(" ", "_")
            if key in positioning_data:
                return json.dumps(positioning_data[key], indent=2)
            return f"No positioning data for {space}"
        
        return [
            Tool(
                name="get_competitor_profiles",
                description="Get competitor profiles, market share, and competitive landscape",
                func=get_competitor_profiles
            ),
            Tool(
                name="get_positioning_analysis",
                description="Get market positioning, differentiation factors, and opportunities",
                func=get_positioning_analysis
            )
        ]
    
    def _initialize_agent(self):
        """Initialize competitive intelligence agent"""
        
        system_prompt = """You are a Competitive Intelligence Agent specializing in competitive landscape analysis and strategic positioning.
        
Your expertise covers:
- Analyzing competitive strengths and weaknesses
- Identifying market positioning opportunities
- Assessing competitive threats and responses
- Understanding differentiation strategies
- Evaluating market entry and expansion strategies

When analyzing competitive data:
1. Assess the competitive landscape structure and dynamics
2. Identify key differentiators and competitive advantages
3. Analyze competitive threats and market positioning
4. Look for white space opportunities and strategic gaps
5. Consider competitive responses and market evolution

Integrate insights from market and consumer research to provide comprehensive competitive perspective."""
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        agent = create_openai_functions_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=prompt
        )
        
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=3
        )
    
    def conduct_analysis(self, query: str) -> str:
        """Conduct competitive analysis"""
        # Get context from other agents
        context = self.shared_kb.get_findings_for_agent(self.name)
        
        enhanced_query = f"""{query}
        
Context from market and consumer research:
{context}

Please provide comprehensive competitive analysis and strategic recommendations."""
        
        try:
            result = self.agent_executor.invoke({"input": enhanced_query})
            
            # Store findings
            self.shared_kb.add_finding(
                "competitive",
                query[:50].replace(" ", "_"),
                {"summary": result["output"][:200] + "...", "full_analysis": result["output"]},
                self.name
            )
            
            return result["output"]
        except Exception as e:
            return f"Competitive analysis encountered an issue: {str(e)}"

# Initialize Competitive Intelligence Agent
competitive_agent = CompetitiveIntelligenceAgent(openai_api_key, shared_kb)
print("Competitive Intelligence Agent ready - will perform actual competitive analysis")

Competitive Intelligence Agent ready - will perform actual competitive analysis


In [32]:
class ResearchCoordinator:
    """Orchestrates multi-agent research and synthesizes findings"""
    
    def __init__(self, openai_api_key: str, shared_kb: SharedKnowledgeBase):
        self.shared_kb = shared_kb
        self.agents = {}
        self.sessions = {}
        
        self.llm = ChatOpenAI(
            temperature=0.3,  # More focused for synthesis
            api_key=openai_api_key,
            model_name="gpt-4"
        )
        
    def register_agent(self, name: str, agent):
        """Register agents for coordination"""
        self.agents[name] = agent
    
    def create_session(self, topic: str, objectives: List[str]) -> str:
        """Create research session"""
        session_id = str(uuid.uuid4())[:8]
        self.sessions[session_id] = {
            "topic": topic,
            "objectives": objectives,
            "start_time": datetime.now(),
            "status": "active"
        }
        return session_id
    
    def execute_research(self, session_id: str, tasks: Dict[str, List[str]]) -> Dict[str, Any]:
        """Execute coordinated research across agents"""
        session = self.sessions[session_id]
        results = {}
        
        print(f"\n🔬 MULTI-AGENT RESEARCH SESSION")
        print(f"Topic: {session['topic']}")
        print(f"Session ID: {session_id}")
        print("=" * 60)
        
        # Execute tasks in sequence so agents can build on each other
        phase_num = 1
        
        # Market Research Phase
        if "market_research" in tasks and "market_agent" in self.agents:
            print(f"\n📊 PHASE {phase_num}: MARKET RESEARCH ANALYSIS")
            print("=" * 50)
            
            market_results = []
            for task in tasks["market_research"]:
                print(f"\nTask: {task}")
                print("-" * 40)
                result = self.agents["market_agent"].conduct_research(task)
                market_results.append(result)
                print("\nAgent Analysis Complete")
                print("=" * 40)
            
            results["market_analysis"] = market_results
            phase_num += 1
        
        # Consumer Research Phase
        if "consumer_research" in tasks and "consumer_agent" in self.agents:
            print(f"\n👥 PHASE {phase_num}: CONSUMER INSIGHTS ANALYSIS")
            print("=" * 50)
            
            consumer_results = []
            for task in tasks["consumer_research"]:
                print(f"\nTask: {task}")
                print("-" * 40)
                result = self.agents["consumer_agent"].conduct_research(task)
                consumer_results.append(result)
                print("\nAgent Analysis Complete")
                print("=" * 40)
            
            results["consumer_insights"] = consumer_results
            phase_num += 1
        
        # Competitive Analysis Phase
        if "competitive_analysis" in tasks and "competitive_agent" in self.agents:
            print(f"\n🎯 PHASE {phase_num}: COMPETITIVE INTELLIGENCE ANALYSIS")
            print("=" * 50)
            
            competitive_results = []
            for task in tasks["competitive_analysis"]:
                print(f"\nTask: {task}")
                print("-" * 40)
                result = self.agents["competitive_agent"].conduct_analysis(task)
                competitive_results.append(result)
                print("\nAgent Analysis Complete")
                print("=" * 40)
            
            results["competitive_intelligence"] = competitive_results
            phase_num += 1
        
        # Strategic Synthesis Phase
        print(f"\n🧠 PHASE {phase_num}: STRATEGIC SYNTHESIS")
        print("=" * 50)
        
        synthesis = self._synthesize_findings(session_id)
        results["strategic_synthesis"] = synthesis
        
        # Complete session
        session["status"] = "completed"
        session["end_time"] = datetime.now()
        session["results"] = results
        
        print("\n✅ MULTI-AGENT RESEARCH SESSION COMPLETE")
        print("=" * 60)
        return results
    
    def _synthesize_findings(self, session_id: str) -> str:
        """Use LLM to synthesize all agent findings"""
        all_findings = self.shared_kb.get_all_findings()
        
        # Prepare synthesis prompt with all findings
        findings_summary = "\n".join([
            f"Agent: {finding['agent']}\n"
            f"Category: {finding['category']}\n"
            f"Analysis: {finding['data']['full_analysis'][:500]}...\n"
            for finding in all_findings.values()
        ])
        
        synthesis_prompt = f"""As a strategic research coordinator, synthesize the following research findings from multiple AI agents into a comprehensive strategic analysis:

AGENT FINDINGS:
{findings_summary}

Please provide:
1. Key insights from cross-agent analysis
2. Strategic implications and opportunities
3. Risk assessment and mitigation strategies
4. Actionable recommendations
5. Implementation priorities

Focus on insights that emerge from combining multiple agent perspectives."""
        
        try:
            synthesis_result = self.llm.invoke(synthesis_prompt)
            synthesis_content = synthesis_result.content if hasattr(synthesis_result, 'content') else str(synthesis_result)
            
            print("\n🎯 STRATEGIC SYNTHESIS:")
            print("-" * 50)
            print(synthesis_content)
            print("-" * 50)
            
            return synthesis_content
        except Exception as e:
            return f"Synthesis encountered an issue: {str(e)}"
    
    def generate_executive_report(self, session_id: str) -> str:
        """Generate comprehensive executive report"""
        if session_id not in self.sessions:
            return "Session not found"
        
        session = self.sessions[session_id]
        if session["status"] != "completed":
            return "Session not completed"
        
        duration = (session["end_time"] - session["start_time"]).seconds // 60
        
        report = f"""EXECUTIVE RESEARCH REPORT
========================

Research Topic: {session['topic']}
Completion Date: {session['end_time'].strftime('%B %d, %Y at %I:%M %p')}
Research Duration: {duration} minutes
Session ID: {session_id}

RESEARCH METHODOLOGY:
====================
This report synthesizes findings from multiple AI agents conducting specialized analysis:
- Market Research Agent: Industry and market analysis
- Consumer Insights Agent: User behavior and adoption patterns
- Competitive Intelligence Agent: Competitive landscape and positioning
- Research Coordinator: Strategic synthesis and recommendations

EXECUTIVE SUMMARY:
=================
{session['results']['strategic_synthesis']}

AGENT CONTRIBUTIONS:
==================="""
        
        # Add agent contributions summary
        for agent_name, contributions in self.shared_kb.agent_contributions.items():
            report += f"\n\n{agent_name}: {len(contributions)} analyses completed"
        
        report += f"""\n\nMETHODOLOGY NOTES:
=================
• Agents used LLM reasoning to analyze data and generate insights
• Cross-agent collaboration through shared knowledge base
• Sequential analysis allowing agents to build upon previous findings
• Strategic synthesis combining multiple perspectives

Report generated by: Multi-Agent Research System
========================"""
        
        return report

# Initialize Research Coordinator
coordinator = ResearchCoordinator(openai_api_key, shared_kb)
coordinator.register_agent("market_agent", market_agent)
coordinator.register_agent("consumer_agent", consumer_agent)
coordinator.register_agent("competitive_agent", competitive_agent)

print(f"Research Coordinator ready with {len(coordinator.agents)} agents for real collaboration")

Research Coordinator ready with 3 agents for real collaboration


In [33]:
def run_real_agent_demonstration():
    """Run demonstration showing actual agent reasoning and collaboration"""
    
    print("\n🚀 REAL MULTI-AGENT RESEARCH DEMONSTRATION")
    print("Agents will perform actual analysis and reasoning")
    print("=" * 60)
    
    # Create research session
    topic = "AI Software Market Entry Strategy"
    objectives = [
        "Analyze market dynamics and growth opportunities",
        "Understand consumer adoption patterns and preferences",
        "Assess competitive landscape and positioning opportunities",
        "Generate strategic recommendations for market entry"
    ]
    
    session_id = coordinator.create_session(topic, objectives)
    print(f"Research Session Created: {session_id}")
    
    # Define research tasks for agents
    research_tasks = {
        "market_research": [
            "Analyze the AI software market size, growth projections, and key market segments. What are the primary growth drivers and regional opportunities?",
            "Examine current market trends and emerging technologies in the AI software space. What shifts are occurring and what implications do they have?"
        ],
        "consumer_research": [
            "Analyze consumer adoption patterns for AI software across different market segments. What drives adoption and what are the barriers?",
            "Examine user behavior, preferences, and decision-making factors. What do users prioritize when selecting AI software solutions?"
        ],
        "competitive_analysis": [
            "Analyze the competitive landscape in AI software. Who are the key players and how is the market positioned?",
            "Identify white space opportunities and differentiation strategies. Where are the gaps in the current market?"
        ]
    }
    
    # Execute coordinated research
    results = coordinator.execute_research(session_id, research_tasks)
    
    # Generate executive report
    print("\n📊 GENERATING EXECUTIVE REPORT")
    print("=" * 60)
    
    report = coordinator.generate_executive_report(session_id)
    print(report)
    
    return results, report

# Execute the real agent demonstration
demo_results, executive_report = run_real_agent_demonstration()


🚀 REAL MULTI-AGENT RESEARCH DEMONSTRATION
Agents will perform actual analysis and reasoning
Research Session Created: 94ddf5cc

🔬 MULTI-AGENT RESEARCH SESSION
Topic: AI Software Market Entry Strategy
Session ID: 94ddf5cc

📊 PHASE 1: MARKET RESEARCH ANALYSIS

Task: Analyze the AI software market size, growth projections, and key market segments. What are the primary growth drivers and regional opportunities?
----------------------------------------


> Entering new AgentExecutor chain...

Invoking: `get_market_data` with `AI software`


{
  "current_market_size_billion": 62.5,
  "projected_size_2030_billion": 1400,
  "key_segments": [
    "Machine Learning Platforms",
    "Natural Language Processing",
    "Computer Vision",
    "Robotic Process Automation"
  ],
  "regional_breakdown": {
    "North America": 0.42,
    "Europe": 0.28,
    "Asia Pacific": 0.22,
    "Other": 0.08
  },
  "adoption_indicators": [
    "Enterprise digital transformation initiatives",
    "Remote work accelera

In [34]:
# Display final collaboration summary
print("\n🤝 AGENT COLLABORATION SUMMARY")
print("=" * 50)

print(f"\nKnowledge Base Entries: {len(shared_kb.get_all_findings())}")
print("\nAgent Contributions:")
for agent, contributions in shared_kb.agent_contributions.items():
    print(f"  • {agent}: {len(contributions)} findings")

print("\nDemonstration Summary:")
print("• Agents performed actual LLM-based reasoning and analysis")
print("• Real-time knowledge sharing between agents")
print("• Cross-agent collaboration and context building")
print("• Strategic synthesis combining multiple perspectives")
print("• No hardcoded responses - all analysis generated dynamically")

print("\n✅ Real Multi-Agent Research Demonstration Complete!")


🤝 AGENT COLLABORATION SUMMARY

Knowledge Base Entries: 6

Agent Contributions:
  • Market Research Agent: 2 findings
  • Consumer Insights Agent: 2 findings
  • Competitive Intelligence Agent: 2 findings

Demonstration Summary:
• Agents performed actual LLM-based reasoning and analysis
• Real-time knowledge sharing between agents
• Cross-agent collaboration and context building
• Strategic synthesis combining multiple perspectives
• No hardcoded responses - all analysis generated dynamically

✅ Real Multi-Agent Research Demonstration Complete!
